In [16]:
import pandas as pd
import numpy as np
import re

In [17]:
df = pd.read_csv('../../data/raw/century21_v2.csv')

In [18]:
df.isnull().sum()

id                0
information       0
type              0
province          0
price             2
land_area        26
price_per_m2    169
lat               0
long              0
dtype: int64

In [19]:
df = df[df['province'] == 'Phnom Penh']

In [20]:
df['land_area'] = df['land_area'].replace(0.0, np.nan).replace('', np.nan)

In [21]:
# def extract_land_area(text):
#     if pd.isna(text):
#         return None
    
#     # Step 1: Direct land area extraction (e.g., "100 sqm")
#     direct_pattern = r'(\d+\.?\d*)\s*(?:sqm|m²|m2|square\s*meters?)\b'
#     direct_match = re.search(direct_pattern, text, re.IGNORECASE)
#     if direct_match:
#         return float(direct_match.group(1))
    
#     # Step 2: Extract land dimensions (e.g., "4m x 20m") with contextual keywords
#     dim_pattern = r'(\d+\.?\d*)\s*(?:m\s*)?[x*]\s*(\d+\.?\d*)\s*(?:m\b)'
#     dim_matches = list(re.finditer(dim_pattern, text, re.IGNORECASE))
    
#     for match in dim_matches:
#         start, end = match.span()
#         window = text[max(0, start-30):min(len(text), end+30)].lower()
#         # Check for contextual keywords near the dimensions
#         if re.search(r'\b(?:land|plot|ground|property|size)\b', window):
#             length = float(match.group(1))
#             width = float(match.group(2))
#             return length * width
    
#     # Step 3: Fallback to the first dimension pattern found (if any)
#     if dim_matches:
#         length = float(dim_matches[0].group(1))
#         width = float(dim_matches[0].group(2))
#         return length * width
    
#     return None

In [22]:
def extract_land_area(text):
    if pd.isna(text):
        return None
    def parse_float_number(num_str):
        if num_str is None:
            return None
        return float(num_str.replace(',', '.'))
    number_pattern = r'\d+(?:[.,]\d*)?'

    # Step 1: Direct land area extraction (e.g., "100 sqm", "9,2 m2")
    direct_pattern = rf'({number_pattern})\s*(?:sqm|m²|m2|square\s*meters?)\b'
    direct_match = re.search(direct_pattern, text, re.IGNORECASE)
    if direct_match:
        extracted_area = parse_float_number(direct_match.group(1))
        # Apply the condition: if extracted_area < 30, return None
        if extracted_area is not None and extracted_area < 30:
            return None
        return extracted_area
    
    # Step 2: Extract land dimensions (e.g., "4m x 20m", "9,5m*10m") with contextual keywords
    dim_pattern = rf'({number_pattern})\s*(?:m\s*)?[x*]\s*({number_pattern})\s*(?:m\b)'
    dim_matches = list(re.finditer(dim_pattern, text, re.IGNORECASE))
    
    for match in dim_matches:
        start, end = match.span()
        window = text[max(0, start-30):min(len(text), end+30)].lower()
        
        if re.search(r'\b(?:land|plot|ground|property|size)\b', window):
            length = parse_float_number(match.group(1))
            width = parse_float_number(match.group(2))
            
            if length is not None and width is not None:
                extracted_area = length * width
                # Apply the condition: if extracted_area < 30, return None
                if extracted_area < 30:
                    return None
                return extracted_area
    
    # Step 3: Fallback to the first dimension pattern found (if any)
    if dim_matches:
        length = parse_float_number(dim_matches[0].group(1))
        width = parse_float_number(dim_matches[0].group(2))
        
        if length is not None and width is not None:
            extracted_area = length * width
            # Apply the condition: if extracted_area < 30, return None
            if extracted_area < 30:
                return None
            return extracted_area
    
    return None

In [23]:
rows_to_reprocess = df['land_area'].apply(
    lambda x: pd.isna(x) or (pd.api.types.is_numeric_dtype(x) and (x == 0 or x < 30))
)

# Iterate only over the rows that meet the condition
for idx, row in df[rows_to_reprocess].iterrows():
    extracted_area = extract_land_area(row['information'])
    
    if extracted_area is not None:
        df.at[idx, 'land_area'] = extracted_area
    else:
        df.at[idx, 'land_area'] = np.nan

In [24]:
df.loc[df['land_area'].apply(lambda x: pd.api.types.is_numeric_dtype(x) and x < 30), 'land_area'] = np.nan

In [25]:
c =0
for i, row in df.iterrows():
    
    if row['land_area'] < 30:
        row['land_area'] = np.nan

In [26]:
c =0
for i, row in df.iterrows():
    
    if row['land_area'] < 30:
        c += 1
print(c)

0


In [27]:
# df.drop(columns='garages', inplace=True)

In [28]:
# indexPrice = df[(df['price_display'] == 'POA')].index
# df.drop(indexPrice, inplace=True)

In [29]:
# indexPrice = df[(df['price_display']<df['price_display'])].index
# # df.drop(indexPrice, inplace=True)

In [30]:
# print(indexPrice)

In [35]:
df.isnull().sum()

id               0
information      0
type             0
province         0
price            2
land_area       26
price_per_m2    27
lat              0
long             0
dtype: int64

In [33]:
df.loc[df['price_per_m2'].isnull(), 'price_per_m2'] = df['price'] / df['land_area']

In [36]:
df.dropna(inplace=True)

In [37]:
df.to_csv('../../data/raw/century21_v2.csv', index=False)